# Notebook to calculate statistics about circuits

In [1]:
import warnings
import json
import os
import sys
import glob
import numpy
import pickle
import torch
import random
import pennylane as qml
from discopy.quantum.pennylane import to_pennylane

from math import ceil
from pathlib import Path
import numpy as np
from sympy import default_sort_key
from utils import *

this_folder = os.path.abspath(os.getcwd())

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//"\
            + workload + "//" + workload_size + "//circuits//"\
            + str(classification) + "//" + str(layers) + "_layer//"\
            + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count)\
            + "_n_wire_count//"

In [3]:
training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [4]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

In [5]:
training_data = None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
    
labels, stats = create_labeled_classes(training_data, classification, workload, True)

Collected statistics:
- Classification task
- Number of classes
- Number of training circuits
- Number of validation circuits
- Number of test circuits
- Total number of circuits
- Total number of variables
- Single qubit gates in total:
    - Hadamard
    - Rx
     - Rz
- Two-qubit gates
    - Ctrl-Rz
- Avarage depth of circuits
- Avarage number of qubits of circuits
- Classes i.e. thresholds
- KL(P || Q) i.e. 0.017 nats

In [6]:
def get_pennylane_circuit(disco_circuit):
    qml_circuit = to_pennylane(disco_circuit)
    qnode_circuit = qml_circuit.make_circuit()
    current_symbols = list(sorted(disco_circuit.free_symbols, key=default_sort_key))
    params = torch.Tensor([[2*np.pi*random.uniform(0, 1)] for i in range(len(current_symbols))])
    params = qml_circuit.param_substitution(current_symbols, params)
    return qnode_circuit, params, current_symbols

In [7]:
numpy.random.seed(0)
rng = numpy.random.default_rng(0)
result = {}
train_symbols = set()
total_depth = 0
total_qubits = 0

for c in training_circuits:
    #init_params = np.array(rng.random(len(train_symbols)))
    disco_circuit = training_circuits[c]
    qml_circuit, params, current_symbols = get_pennylane_circuit(disco_circuit)
    train_symbols |= set(current_symbols)
    specs_func = qml.specs(qml_circuit)
    executed_circuit = specs_func(params)
    total_depth += executed_circuit["depth"]
    total_qubits += executed_circuit["num_used_wires"]
    gate_types = executed_circuit["gate_types"]
    for elem in gate_types:
        if elem not in result:
            result[elem] = gate_types[elem]
        else:
            result[elem] = result[elem] + gate_types[elem]

In [8]:
with open('results/statistics.json', 'r') as openfile:
    statistics = json.load(openfile)

key = workload + "_" + workload_size + "_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params) + "_" + str(n_wire_count)

class_ranges = ""
for r in stats:
    class_ranges += "[" + str(r[0]) + "," + str(r[1]) + "];"
    
statistics[key] = { "workload": workload, 
                    "classification" : 2**classification,
                    "class_ranges": class_ranges,
                    "training_circuits": len(training_circuits), 
                    "validation_circuits": len(validation_circuits), 
                    "test_circuits": len(test_circuits), 
                    "total": len(training_circuits)+len(validation_circuits)+len(test_circuits), 
                    "train_symbols": len(train_symbols),
                    "avarage_depth": total_depth/len(training_circuits),
                    "avarage_number_of_qubits": total_qubits/len(training_circuits),
                    "gate_counts": result }

with open('results/statistics.json', "w") as outfile:
    json.dump(statistics, outfile)

In [9]:
def json_to_latex_table(json):
    latex_beginning = """
    \\begin{table*}[t]
        \centering
    \\begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|}
    \hline"""
    
    table_attributes = """
    Task & 
    \#classes &
    ranges &
    \#parameters &
    avg. depth &
    avg. qubits &
    \multicolumn{3}{|c|}{Single qubit gates} & 
    Two-qubit & 
    Results \\\\
    & & & & & & \#H & \#Rx & \#Rz & \#CRz & \\\\
    \hline
    """
    
    data = ""
    for e in json:
        for element in json[e]:
            if type(json[e][element]) == str or type(json[e][element]) == float or type(json[e][element]) == int:
                data += str(json[e][element]) + " & "
            else:
                for elem in ["Hadamard", "RX", "RZ", "CRZ"]:
                    data += str(json[e][element][elem]) + " & "
        data += "\\\\"
    
    latex_ending = """
    \\hline
    \end{tabular}
    \label{tab:statistics}
    \end{table*}
    """
    return latex_beginning + table_attributes + data + latex_ending

In [10]:
with open('results/statistics.json', 'r') as openfile:
    statistics = json.load(openfile)
    print(json_to_latex_table(statistics))


    \begin{table*}[t]
        \centering
    \begin{tabular}{|c|c|c|c|c|c|c|c|c|c|c|}
    \hline
    Task & 
    \#classes &
    ranges &
    \#parameters &
    avg. depth &
    avg. qubits &
    \multicolumn{3}{|c|}{Single qubit gates} & 
    Two-qubit & 
    Results \\
    & & & & & & \#H & \#Rx & \#Rz & \#CRz & \\
    \hline
    execution_time & 2 & [0.0929,263.9624];[263.9903,15603.7168]; & 448 & 113 & 112 & 673 & 286 & 30.0625 & 9.564732142857142 & 7202 & 26066 & 4354 & 3837 & \\cardinality & 2 & [0,33181];[33181,102540525]; & 448 & 113 & 112 & 673 & 283 & 30.0625 & 9.564732142857142 & 7202 & 25170 & 3906 & 3837 & \\cardinality & 4 & 448 & 113 & 112 & 673 & 279 & 20.0625 & 10.564732142857142 & 7650 & 22482 & 3906 & 4285 & \\execution_time & 4 & [0.0929,197.3973];[197.6023,263.9903];[264.8415,915.3311];[916.3591,15603.7168]; & 448 & 113 & 112 & 673 & 282 & 20.0625 & 10.564732142857142 & 7650 & 23378 & 4354 & 4285 & \\
    \hline
    \end{tabular}
    \label{tab:statistics}
    \en